In [70]:
import numpy as np
import pandas as pd
import math
import glob
from astroquery.ipac.irsa import Irsa
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.simbad import Simbad
from astropy.table import Table


In [71]:
#import the data frames. rewrote some of the columns to make them the way i wanted.
#testing
gsample = pd.read_csv('../Data/galaxy_names_reduced_homogeneous_resolved_unique_with_10ks_exposure_cut.csv')
#print(gsample)
gsample = pd.DataFrame(gsample['Galaxy_Name_Reduced'][:])

gsample = gsample.rename(columns={'Galaxy_Name_Reduced': 'source_id'})
gsample['source_id']
#print(gsample)
#gsample.to_csv('output1.csv', index=True)

oldsample = pd.read_csv('../Data/ocatResult_Modified.csv')
oldsample
#gsample

,Unnamed: 0,Seq Num,Obs ID,Instrument,Grating,Appr Exp,Exposure,Target Name,PI Name,RA,...,Evt Cnt,Start Date,Public Release Date,Proposal,Science Category,Type,Obs Cycle,Prop Cycle,Joint,Gname
0,0,600076,316,ACIS-I,NONE,37.0,35.72,NGC 5128,Murray,201.365,...,247153.0,12/5/1999 21:36,12/30/2000 9:30,1600065,NORMAL GALAXIES,GTO,1,1,NaN,NGC 5128
1,1,600078,318,ACIS-S,NONE,14.5,14.63,NGC 4406,Murray,186.550,...,212449.0,4/7/2000 7:36,4/20/2001 16:20,1600133,NORMAL GALAXIES,GTO,1,1,NaN,MESSIER 086
2,2,700021,349,ACIS-S,NONE,1.0,1.57,NGC 4258,Wilson,184.741,...,2194.0,3/8/2000 2:23,3/23/2001 10:40,1700035,ACTIVE GALAXIES AND QUASARS,GTO,1,1,NaN,MESSIER 106
3,3,700022,350,ACIS-S,NONE,14.5,14.04,NGC 4258,Wilson,184.741,...,164036.0,4/17/2000 10:51,7/17/2001 9:30,1700035,ACTIVE GALAXIES AND QUASARS,GTO,1,1,NaN,MESSIER 106
4,4,700025,353,ACIS-S,NONE,1.0,0.82,M51,Wilson,202.468,...,822.0,3/21/2000 17:11,4/18/2001 18:30,1700035,ACTIVE GALAXIES AND QUASARS,GTO,1,1,NaN,MESSIER 051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,720,601545,25782,ACIS-S,NONE,30.0,NaN,ngc 4631,Hodges-Kluck,190.533,...,NaN,NaN,NaN,23610025,NORMAL GALAXIES,GO,23,23,NaN,NGC 4631
721,721,601539,25989,ACIS-S,NONE,17.0,16.73,NGC 3998,Zepf,179.484,...,219003.0,9/7/2021 23:42,NaN,22620521,NORMAL GALAXIES,GO,22,22,NaN,NGC 3998
722,722,601539,25990,ACIS-S,NONE,14.0,13.77,NGC 3998,Zepf,179.484,...,181613.0,9/12/2021 17:26,NaN,22620521,NORMAL GALAXIES,GO,22,22,NaN,NGC 3998
723,723,704563,26038,ACIS-I,NONE,5.0,NaN,Centaurus A,Kraft,201.365,...,NaN,NaN,NaN,23700662,ACTIVE GALAXIES AND QUASARS,GTO,23,23,NaN,NGC 5128


In [72]:
# I want only a specific few columns just to make it simpler

columns = ['RA','Dec','Gname']
name_and_coords = oldsample[columns]
name_and_coords

,RA,Dec,Gname
0,201.365,-43.019,NGC 5128
1,186.550,12.950,MESSIER 086
2,184.741,47.303,MESSIER 106
3,184.741,47.303,MESSIER 106
4,202.468,47.198,MESSIER 051
...,...,...,...
720,190.533,32.542,NGC 4631
721,179.484,55.454,NGC 3998
722,179.484,55.454,NGC 3998
723,201.365,-43.019,NGC 5128


In [73]:
gsample['source_id']

0         NGC 5128
1      MESSIER 086
2      MESSIER 106
3      MESSIER 051
4      MESSIER 082
          ...     
116       NGC 4302
117       NGC 1792
118          IC342
119       NGC 3287
120    MESSIER 096
Name: source_id, Length: 121, dtype: object

# Querying Simbad



In [74]:

# this block is useful for doing simbad queries but does not really help. some interesting functions like "to list"

sample_list = gsample['source_id'].tolist()
#print(sample_list)
#Set simbad settings to retrieve Ra and Dec
simbad_settings = Simbad()
#simbad_settings.add_votable_feilds("coordinates")
#testing to make sure querying simbad works
test= Simbad.query_object("M1")

#querying simbad using the list running over each name in the list and pulling out all the information on it.
#i specifically wanted to see the RA and DEC but i'll get many other things from it and that's ok
myresult= []

for galaxy in sample_list:
    result = Simbad.query_object(galaxy)
    myresult.append(result)

#myresult

# Creating table to upload to ALLWISE catalog

In [75]:
# the code that Katie helped me with. Not something that i should be practicing since apparently you should not
# should not use for loops with pandas since the reason it exists is to eliminate the need for them
# nonetheless, this uses names from the list of samples above and goes into "name_and_coords"(which is a variable for
# a column reduced csv file that Anthony gave us with all the galaxies in the sample and had way too many duplicates)
# and finds the location (.loc) for any name that matches what is in the small csv file with the big csv file and appends it
# as you can see it reduces the number somewhat cuz of the 10ks cut, but there are a lot of duplicates

targets = sample_list
columns = name_and_coords.columns
#create empty data frame
target_data = pd.DataFrame(columns=columns)
for target in targets:
    row = name_and_coords.loc[name_and_coords['Gname']==target]
    target_data = target_data._append(row)


target_data


/var/folders/nd/77pj43rx7wbcbp0wx2qjq2140000gp/T/ipykernel_14463/303889620.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  target_data = target_data._append(row)


,RA,Dec,Gname
0,201.365,-43.019,NGC 5128
47,201.365,-43.019,NGC 5128
104,201.370,-43.017,NGC 5128
131,201.370,-43.017,NGC 5128
272,201.330,-43.045,NGC 5128
...,...,...,...
702,161.691,11.820,MESSIER 096
703,161.691,11.820,MESSIER 096
704,161.691,11.820,MESSIER 096
706,161.691,11.820,MESSIER 096


In [76]:
# removes the duplicates
target_data.drop_duplicates(subset = ['Gname'])

,RA,Dec,Gname
0,201.365,-43.019,NGC 5128
1,186.550,12.950,MESSIER 086
2,184.741,47.303,MESSIER 106
4,202.468,47.198,MESSIER 051
6,148.963,69.679,MESSIER 082
...,...,...,...
578,185.427,14.598,NGC 4302
595,76.310,-37.981,NGC 1792
645,56.702,68.096,IC342
694,158.702,21.662,NGC 3287


In [77]:
#wanted to rename the columns again cuz i was fighting with the document upload feature on WISE telescope

target_data = target_data.rename(columns={'Gname': 'source_id'})
target_data = target_data.rename(columns={'RA': 'ra'})
target_data = target_data.rename(columns={'Dec': 'dec'})

target_data = target_data.drop_duplicates(subset = ['source_id'])

In [78]:
target_data

,ra,dec,source_id
0,201.365,-43.019,NGC 5128
1,186.550,12.950,MESSIER 086
2,184.741,47.303,MESSIER 106
4,202.468,47.198,MESSIER 051
6,148.963,69.679,MESSIER 082
...,...,...,...
578,185.427,14.598,NGC 4302
595,76.310,-37.981,NGC 1792
645,56.702,68.096,IC342
694,158.702,21.662,NGC 3287


In [79]:
targetnames = target_data

In [80]:
#write each row as separated by |
#with open('../Data/file_ascii.txt', 'w', encoding = 'ascii', errors = 'ignore') as f:
   # for index, row in target_data:
     #   row_string = '|'.join(map(str, row.values)) + '\n'
       # f.write(row_string)

# still fighting but i'll leave it here just in case   
    
columns1 = ['ra','dec']
target_data = target_data[columns1]
len(target_data)

120

In [81]:
#target_data.to_csv('final1.csv', index=False, sep = ' ')

In [82]:

#a nice thing i found on stack overflow for creating a .tbl file! writing it into one of the formats that WISE only accepts

data_out = Table.from_pandas(target_data)
output_path = '../Data/data_out.tbl'

# Write the table in IPAC format
data_out.write(output_path, format='ipac')


OSError: File ../Data/data_out.tbl already exists. If you mean to replace it then use the argument "overwrite=True".

In [83]:
# I want a csv version of this too
data_out = Table.from_pandas(target_data)
output_path = '../Data/data_outs.csv'

# Write the table in csv format
data_out.write(output_path, format='csv')



OSError: File ../Data/data_outs.csv already exists. If you mean to replace it then use the argument "overwrite=True".

# Querying ALLWISE

In [84]:
#querying the ALLWISE database

#trying to figure out the names of columns so i know what to 
#catalogs = Irsa.list_catalogs()
#print(target_data)
for index, row in target_data.iterrows():
    ra = row['ra']
    dec = row['dec']
    coordinates= SkyCoord(ra,dec, unit= (u.deg, u.deg), frame= 'icrs' )
    table = Irsa.query_region(coordinates, catalog = "allwise_p3as_psd", spatial="Cone")
#i think it works but it is taking too long on my mac


#print("All catalogs", catalogs)
#table = Irsa.query_region("HIP 12", catalog="allwise_p3as_psd", spatial="Cone")
#print(table)

KeyboardInterrupt: 

In [85]:
print(table)

designation  ra dec sigra  sigdec sigradec ...  x   y   z  spt_ind htm20
            deg deg arcsec arcsec  arcsec  ...                          
----------- --- --- ------ ------ -------- ... --- --- --- ------- -----


# Reading in results from the ALLWISE Catalog table
    Also figuring out the fraction of galaxies present and which ones are not present


In [89]:
#data = np.genfromtxt('../Data/table_irsa_catalog_search_results.tbl')

# found out that you can change the type of document you can download from WISE which helped a lot (csv files are goated)
data = pd.read_csv('../Data/table_irsa_catalog_search_results.csv')
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
data

#THE DESCRIPTIONS FOR EACH COLUMN IN THE DATA TABLE CAN BE SEEN IN THE DOWNLOADED FILE!! i think you have a version of it as a tbl which is a little more legible.
# I'm not sure why, but for some reason there are some repeats. if you look closely at the Ra_01 and Dec_01 returned
# i think it duplicates the values almost exactly as many times as there are missing ones.

,cntr_01,dist_x,pang_x,ra_01,dec_01,designation,ra,dec,sigra,sigdec,sigradec,w1mpro,w1sigmpro,w1snr,w1rchi2,w2mpro,w2sigmpro,w2snr,w2rchi2,w3mpro,w3sigmpro,w3snr,w3rchi2,w4mpro,w4sigmpro,w4snr,w4rchi2,nb,na,w1sat,w2sat,w3sat,w4sat,pmra,sigpmra,pmdec,sigpmdec,cc_flags,ext_flg,var_flg,ph_qual,moon_lev,w1nm,w1m,w2nm,w2m,w3nm,w3m,w4nm,w4m
0,1,0.830776,162.673944,201.365,-43.019,J132527.62-430109.1,201.365094,-43.019220,0.0263,0.0252,-0.0031,5.262,0.051,21.1,19.1000,4.757,0.041,26.4,12.2300,2.621,0.012,89.4,20.4300,0.279,0.015,71.3,18.9800,1,0,0.100,0.097,0.075,0.000,-65,22,-305,22,hhhh,5,3344,AAAA,0,46,46,46,46,24,24,24,24
1,3,5.022737,-43.454463,184.741,47.303,J121857.50+471814.4,184.739585,47.304013,0.0301,0.0302,-0.0049,8.530,0.023,47.4,50.1600,8.190,0.020,53.9,27.5600,5.482,0.015,70.8,9.1490,3.328,0.020,53.2,5.9670,1,0,0.000,0.000,0.000,0.000,218,47,-347,60,0000,5,2211,AAAA,0,34,34,34,34,19,19,19,19
2,4,6.295556,-65.412743,202.468,47.198,J132951.75+471155.4,202.465660,47.198728,0.0304,0.0294,-0.0027,9.855,0.023,46.9,123.7000,9.661,0.021,52.7,86.9200,5.366,0.013,83.2,114.0000,2.931,0.027,40.0,57.7100,3,0,0.000,0.000,0.000,0.000,326,32,-1422,33,hh00,3,n3n4,AAAA,0,32,32,32,32,18,18,18,18
3,5,4.934001,60.594118,148.963,69.679,J095551.94+694046.8,148.966438,69.679673,0.0014,0.0014,-0.0003,3.669,0.383,2.8,0.2359,3.286,0.053,20.6,15.6600,-1.666,0.321,3.4,0.1714,-4.348,0.001,1253.0,5886.0000,1,0,0.219,0.161,0.328,0.441,-26,31,-107,30,hhhh,5,130n,CABA,0,39,39,39,39,19,19,19,19
4,5,9.488868,61.783241,148.963,69.679,J095552.72+694048.8,148.969688,69.680246,0.0033,0.0032,0.0014,3.953,0.196,5.5,0.9741,4.081,0.012,88.3,377.5000,-1.304,0.369,2.9,0.2508,-3.385,0.002,481.6,13040.0000,1,0,0.205,0.145,0.327,0.434,9254,21,1606,21,dddH,5,4n2n,BACA,0,32,40,40,40,18,20,18,18
5,6,8.361842,-51.691199,67.704,64.848,J043047.93+645057.9,67.699712,64.849440,0.0427,0.0393,-0.0248,9.450,0.028,38.3,442.4000,9.095,0.026,42.3,324.4000,5.004,0.017,65.3,75.3800,0.956,0.016,69.1,87.6300,3,0,0.000,0.000,0.000,0.000,-5809,42,-200,37,hhdd,3,nn23,AAAA,0,33,33,33,33,17,17,17,17
6,8,0.993958,158.563643,11.888,-25.288,J004733.14-251717.7,11.888112,-25.288257,0.0049,0.0039,0.0010,5.757,0.034,32.3,44.2100,5.028,0.029,37.4,33.4000,0.087,0.024,45.5,0.2416,-2.656,0.002,515.7,371.4000,1,0,0.124,0.124,0.308,0.249,-1351,37,539,33,HHHH,5,nnnn,AAAA,0,27,27,27,27,14,14,14,14
7,9,3.890359,103.369259,157.183,68.469,J102844.60+682807.5,157.185865,68.468750,0.0730,0.0756,0.0148,14.595,0.033,33.3,4.5940,14.576,0.047,22.9,1.0900,12.722,NaN,-0.1,0.9390,9.066,0.461,2.4,1.0040,2,0,0.000,0.000,0.000,0.002,0,150,0,160,0000,1,00nn,AAUC,0,36,36,33,36,0,18,0,19
8,10,3.895011,-134.133495,35.639,42.349,J022233.10+422053.6,35.637949,42.348247,0.0296,0.0289,-0.0054,8.741,0.021,52.3,170.3000,8.161,0.019,56.2,103.7000,5.084,0.015,71.8,58.2400,2.955,0.026,41.6,52.9100,2,0,0.000,0.000,0.000,0.000,-1700,27,-2463,26,hhhd,5,nn11,AAAA,0,27,27,27,27,12,12,12,12
9,11,7.368795,-164.984930,49.328,-41.106,J031718.55-410628.7,49.327296,-41.107977,0.0294,0.0288,0.0050,7.194,0.027,40.4,68.6300,7.720,0.019,56.2,56.4700,6.691,0.015,72.3,26.1800,5.091,0.031,35.3,3.1170,1,0,0.042,0.000,0.000,0.000,-131,22,-45,22,0000,5,3331,AAAA,0,65,65,65,65,33,33,33,33


In [90]:

# for some reason the "merge" function wants the columns in both dataframes to be named the exact same
# so this makes things simpler
gcolumns= ['ra_01','dec_01', 'cntr_01']
simple_data = data[gcolumns]
simple_data = simple_data.rename(columns={'ra_01': 'ra'})
simple_data = simple_data.rename(columns={'dec_01': 'dec'})
simple_data

,ra,dec,cntr_01
0,201.365,-43.019,1
1,184.741,47.303,3
2,202.468,47.198,4
3,148.963,69.679,5
4,148.963,69.679,5
5,67.704,64.848,6
6,11.888,-25.288,8
7,157.183,68.469,9
8,35.639,42.349,10
9,49.328,-41.106,11


# merging the data sets

In [91]:
#useful code from katie. not sure what "inner" means, maybe that it doesnt prioritize one dataframe over another.
merged_data = pd.merge(df1, df2, on='the column you want to match', how='inner')
merged_data = merged_data.drop_duplicates()
#merged_data

NameError: name 'df1' is not defined

In [92]:
#using merge to find which of the 114 galaxies found in the WISE telescope are in the list that Anthony gave us. Used a "left merge"
df_all = targetnames.merge(simple_data.drop_duplicates(subset=['cntr_01']), on=['ra', 'dec'], 
                   how='left', indicator=True)
pd.set_option("display.max_rows", None)
df_all[0:122]
#this ra and dec (11.888	-25.288	)is number 6 in this one and 7 in the other?? not good, that's why i'm seeing way too many 'left_only'
# when i put in some of the one's that are left only into the catalogue it comes up with really crappy empty images, but others are also very good (ngc 0404)

,ra,dec,source_id,cntr_01,_merge
0,201.365,-43.019,NGC 5128,1.0,both
1,186.550,12.950,MESSIER 086,NaN,left_only
2,184.741,47.303,MESSIER 106,3.0,both
3,202.468,47.198,MESSIER 051,4.0,both
4,148.963,69.679,MESSIER 082,5.0,both
5,67.704,64.848,NGC 1569,6.0,both
6,192.146,-5.789,NGC 4697,NaN,left_only
7,11.888,-25.288,NGC 0253,8.0,both
8,157.183,68.469,IC 2574,9.0,both
9,35.639,42.349,NGC 0891,10.0,both


In [93]:
df_all = targetnames.merge(data.drop_duplicates(subset=['cntr_01']), left_on=['ra', 'dec'], right_on =['ra_01', 'dec_01'], 
                  how='left', indicator=True)
#pd.set_option("display.max_rows", None)
df_all[0:122]

,ra_x,dec_x,source_id,cntr_01,dist_x,pang_x,ra_01,dec_01,designation,ra_y,dec_y,sigra,sigdec,sigradec,w1mpro,w1sigmpro,w1snr,w1rchi2,w2mpro,w2sigmpro,w2snr,w2rchi2,w3mpro,w3sigmpro,w3snr,w3rchi2,w4mpro,w4sigmpro,w4snr,w4rchi2,nb,na,w1sat,w2sat,w3sat,w4sat,pmra,sigpmra,pmdec,sigpmdec,cc_flags,ext_flg,var_flg,ph_qual,moon_lev,w1nm,w1m,w2nm,w2m,w3nm,w3m,w4nm,w4m,_merge
0,201.365,-43.019,NGC 5128,1.0,0.830776,162.673944,201.365,-43.019,J132527.62-430109.1,201.365094,-43.019220,0.0263,0.0252,-0.0031,5.262,0.051,21.1,19.1000,4.757,0.041,26.4,12.2300,2.621,0.012,89.4,20.4300,0.279,0.015,71.3,18.9800,1.0,0.0,0.100,0.097,0.075,0.000,-65.0,22.0,-305.0,22.0,hhhh,5.0,3344,AAAA,0.0,46.0,46.0,46.0,46.0,24.0,24.0,24.0,24.0,both
1,186.550,12.950,MESSIER 086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,184.741,47.303,MESSIER 106,3.0,5.022737,-43.454463,184.741,47.303,J121857.50+471814.4,184.739585,47.304013,0.0301,0.0302,-0.0049,8.530,0.023,47.4,50.1600,8.190,0.020,53.9,27.5600,5.482,0.015,70.8,9.1490,3.328,0.020,53.2,5.9670,1.0,0.0,0.000,0.000,0.000,0.000,218.0,47.0,-347.0,60.0,0000,5.0,2211,AAAA,0.0,34.0,34.0,34.0,34.0,19.0,19.0,19.0,19.0,both
3,202.468,47.198,MESSIER 051,4.0,6.295556,-65.412743,202.468,47.198,J132951.75+471155.4,202.465660,47.198728,0.0304,0.0294,-0.0027,9.855,0.023,46.9,123.7000,9.661,0.021,52.7,86.9200,5.366,0.013,83.2,114.0000,2.931,0.027,40.0,57.7100,3.0,0.0,0.000,0.000,0.000,0.000,326.0,32.0,-1422.0,33.0,hh00,3.0,n3n4,AAAA,0.0,32.0,32.0,32.0,32.0,18.0,18.0,18.0,18.0,both
4,148.963,69.679,MESSIER 082,5.0,4.934001,60.594118,148.963,69.679,J095551.94+694046.8,148.966438,69.679673,0.0014,0.0014,-0.0003,3.669,0.383,2.8,0.2359,3.286,0.053,20.6,15.6600,-1.666,0.321,3.4,0.1714,-4.348,0.001,1253.0,5886.0000,1.0,0.0,0.219,0.161,0.328,0.441,-26.0,31.0,-107.0,30.0,hhhh,5.0,130n,CABA,0.0,39.0,39.0,39.0,39.0,19.0,19.0,19.0,19.0,both
5,67.704,64.848,NGC 1569,6.0,8.361842,-51.691199,67.704,64.848,J043047.93+645057.9,67.699712,64.849440,0.0427,0.0393,-0.0248,9.450,0.028,38.3,442.4000,9.095,0.026,42.3,324.4000,5.004,0.017,65.3,75.3800,0.956,0.016,69.1,87.6300,3.0,0.0,0.000,0.000,0.000,0.000,-5809.0,42.0,-200.0,37.0,hhdd,3.0,nn23,AAAA,0.0,33.0,33.0,33.0,33.0,17.0,17.0,17.0,17.0,both
6,192.146,-5.789,NGC 4697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,11.888,-25.288,NGC 0253,8.0,0.993958,158.563643,11.888,-25.288,J004733.14-251717.7,11.888112,-25.288257,0.0049,0.0039,0.0010,5.757,0.034,32.3,44.2100,5.028,0.029,37.4,33.4000,0.087,0.024,45.5,0.2416,-2.656,0.002,515.7,371.4000,1.0,0.0,0.124,0.124,0.308,0.249,-1351.0,37.0,539.0,33.0,HHHH,5.0,nnnn,AAAA,0.0,27.0,27.0,27.0,27.0,14.0,14.0,14.0,14.0,both
8,157.183,68.469,IC 2574,9.0,3.890359,103.369259,157.183,68.469,J102844.60+682807.5,157.185865,68.468750,0.0730,0.0756,0.0148,14.595,0.033,33.3,4.5940,14.576,0.047,22.9,1.0900,12.722,NaN,-0.1,0.9390,9.066,0.461,2.4,1.0040,2.0,0.0,0.000,0.000,0.000,0.002,0.0,150.0,0.0,160.0,0000,1.0,00nn,AAUC,0.0,36.0,36.0,33.0,36.0,0.0,18.0,0.0,19.0,both
9,35.639,42.349,NGC 0891,10.0,3.895011,-134.133495,35.639,42.349,J022233.10+422053.6,35.637949,42.348247,0.0296,0.0289,-0.0054,8.741,0.021,52.3,170.3000,8.161,0.019,56.2,103.7000,5.084,0.015,71.8,58.2400,2.955,0.026,41.6,52.9100,2.0,0.0,0.000,0.000,0.000,0.000,-1700.0,27.0,-2463.0,26.0,hhhd,5.0,nn11,AAAA,0.0,27.0,27.0,27.0,27.0,12.0,12.0,12.0,12.0,both


In [94]:
clean_data = df_all.dropna()
print(clean_data)
print(len(clean_data))
print(len(targetnames))
74/120
#61.6 percent of the galaxies are in the WISE dataset

        ra_x   dec_x               source_id  cntr_01    dist_x      pang_x  \
0    201.365 -43.019                NGC 5128      1.0  0.830776  162.673944   
2    184.741  47.303             MESSIER 106      3.0  5.022737  -43.454463   
3    202.468  47.198             MESSIER 051      4.0  6.295556  -65.412743   
4    148.963  69.679             MESSIER 082      5.0  4.934001   60.594118   
5     67.704  64.848                NGC 1569      6.0  8.361842  -51.691199   
7     11.888 -25.288                NGC 0253      8.0  0.993958  158.563643   
9     35.639  42.349                NGC 0891     10.0  3.895011 -134.133495   
10    49.328 -41.106  NGC 1291:[LFF2012] 084     11.0  7.368795 -164.984930   
11   190.530  32.542                NGC 4631     12.0  9.053783   69.702218   
12   192.721  41.120             MESSIER 094     13.0  1.409980   58.249431   
13   196.365 -49.468                NGC 4945     14.0  1.444546  -95.506067   
15   172.005  78.993               UGC 06456     16.

0.6166666666666667

In [95]:
#the galaxies that are missing
#Missing = clean_data.loc[clean_data['cntr_01']['Nan']] # the galaxies that are only in the file that anthony gave us
nan_df = df_all[df_all['cntr_01'].isna()]
obj_nan = nan_df['source_id'].tolist()
print("Galaxies not in the ALLWISE catalog", obj_nan)
#for elem in nan_df['cntr_01']:
    #if elem == True:
        #print(nan_df['source_id'])

#Missing = clean_data.loc[clean_data['cntr_01']['Nan']] 

Galaxies not in the ALLWISE catalog ['MESSIER 086', 'NGC 4697', 'NGC 0404', 'MESSIER 101', 'MESSIER 104', 'NGC 4214', 'NGC 4449', 'NGC 3628', 'NGC 3115', 'NGC 3077', 'NGC 4051', 'NGC 0055', 'NGC 6822', 'NGC 3377', 'NGC 1313', 'NGC 1808', 'NGC 5204', 'NGC 1427A', 'NGC 4565', 'IC 0010', 'MESSIER 095', 'NGC 2841', 'NGC 3998', 'NGC 3621', 'UGC 05423', 'Holmberg I', 'Holmberg IX', 'MESSIER 064', 'NGC 3198', 'NGC 7814', 'NGC 2683', 'NGC 0300', 'Sextans A', 'MESSIER 049', 'NGC 1559', 'NGC 3287']


In [96]:
# I want a csv version of this too
data_out = Table.from_pandas(clean_data)
output_path = '../Data/inWISE.csv'

# Write the table in csv format
data_out.write(output_path, format='csv')


# **Everything below here does not seem to work**

In [ ]:
# hidden code
  #merge_list = df_all['_merge'].tolist()
  #don't remember how to do pull out just the one's that are left_only.
  #for i in merge_list:
    #  if merge_list['_merge'] == True:
        #  print("The galaxy exists in the catalog")

  Target_namesonly = df_all.loc[df_all['_merge'] == 'left_only'] # the galaxies that are only in the file that anthony gave us
  both_dfs = df_all.loc[df_all['_merge'] == 'both']
  #Fraction of galaxies present in the WISE telescope 
  Frac_present = len(Target_namesonly)/len(both_dfs)
  print(Frac_present)
  print 

In [ ]:
# below here I tried to use the OG method Katie and i found but i couldnt get it to work
114/120

In [ ]:
gcolumns= ['ra','dec']
simple_data = data[gcolumns]
simple_data

In [ ]:
targets = simple_data
gcolumns = targetnames.columns
#create empty data frame
missing = pd.DataFrame(columns=gcolumns)
for target in targets:
    row = targetnames.loc[targetnames['ra']==target]
    missing = missing._append(row)

missing
#nothing is showing up because it is not the exact value. you need a partial match, that is something that this does not seem to be able to do.

In [ ]:
data = pd.read_csv('../Data/1to1_irsa_catalog_search_results.csv')
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#data


In [ ]:
#df_all = targetnames.merge(data.drop_duplicates(), left_on=['ra', 'dec'], right_on =['ra_01', 'dec_01'], 
                   how='left', indicator=True)
pd.set_option("display.max_rows", None)
df_all[0:122]


In [ ]:
Missing = df_all.loc[df_all['_merge'] == 'left_only'] # the galaxies that are only in the file that anthony gave us
both_dfs = df_all.loc[df_all['_merge'] == 'both']
#Fraction of galaxies present in the WISE telescope 
Frac_present = len(Missing)/len(both_dfs)
print(Frac_present)
print(len(Missing))


In [ ]:
#nan_df = data.isna()
#print(nan_df)
#nan_rows = nan_df.isna().any(axis=1)
#nan_df
#print(df_all)
#df_all.drop_duplicates(subset = ['cntr_01', 'source_id'])
#print(df_all)
#clean_data = df_all.dropna()

#clean_data